Read the excel file Bird_Monitoring_Data_FOREST.XLSX with all the sheets into a dict using pandas:

In [ ]:
import pandas as pd
# Specify the file path
forest_file_path = "Bird_Monitoring_Data_FOREST.XLSX"

# Read the Excel file with multiple sheets
forest_excel_data = pd.ExcelFile(forest_file_path)

# Get all sheet names
forest_sheet_names = forest_excel_data.sheet_names

# Read data from all sheets into a dictionary
forest_sheets_dict = {sheet: forest_excel_data.parse(sheet) for sheet in forest_sheet_names}
forest_sheets_dict

In [2]:
print(forest_sheets_dict.keys())

dict_keys(['ANTI', 'CATO', 'CHOH', 'GWMP', 'HAFE', 'MANA', 'MONO', 'NACE', 'PRWI', 'ROCR', 'WOTR'])


Now to check if all Forest Sheets have same columns:

In [3]:
# Get list of all DataFrames
dataframes = list(forest_sheets_dict.values())
# Get column names of the first sheet
first_columns = dataframes[0].columns

# Check all others against the first
all_same = all(df.columns.equals(first_columns) for df in dataframes)

print("All sheets have same columns" if all_same else "Sheets have different columns.")



All sheets have same columns


We can now merge the sheets dataframes to a single dataframe called forest_merged_df:

In [4]:
forest_merged_df = pd.concat(dataframes, ignore_index=True)
print(forest_merged_df.shape)


(8546, 29)


Now we get the raw data for Bird_Monitoring_Data_GRASSLAND.XLSX and read all sheets into a dict using pandas:

In [ ]:
import pandas as pd
# Specify the file path
grassland_file_path = "Bird_Monitoring_Data_GRASSLAND.XLSX"

# Read the Excel file with multiple sheets
grassland_excel_data = pd.ExcelFile(grassland_file_path)

# Get all sheet names
grassland_sheet_names = grassland_excel_data.sheet_names

# Read data from all sheets into a dictionary
grassland_sheets_dict = {sheet: grassland_excel_data.parse(sheet) for sheet in grassland_sheet_names}
grassland_sheets_dict

Now we get the Sheets names from the grassland_dict key values

In [6]:
print(grassland_sheets_dict.keys())

dict_keys(['ANTI', 'HAFE', 'MANA', 'MONO'])


Now to check if all Grassland Sheets have same columns:

In [7]:
# Get list of all DataFrames
dataframes = list(grassland_sheets_dict.values())
# Get column names of the first sheet
first_columns = dataframes[0].columns

# Check all others against the first
all_same = all(df.columns.equals(first_columns) for df in dataframes)

print("All sheets have same columns" if all_same else "Sheets have different columns.")

All sheets have same columns


We can now merge the sheets dataframes to a single dataframe called grassland_merged_df:

In [8]:
grassland_merged_df = pd.concat(dataframes, ignore_index=True)
print(grassland_merged_df.shape)

(8531, 29)


The grassland_merged_df has 8531 rows and 29 columns. 

Now we can see if we can merge forest df and grassland df into a single merged_df

In [9]:
import pandas as pd

# Step 1: Check if columns are the same
if forest_merged_df.columns.equals(grassland_merged_df.columns):
    # Step 2: Merge the DataFrames
    merged_df = pd.concat([forest_merged_df, grassland_merged_df], ignore_index=True)
    print("Successfully merged. Shape of merged_df:", merged_df.shape)
else:
    print("Cannot merge: Columns are not the same.")
    print("forest_merged_df columns:", list(forest_merged_df.columns))
    print("grassland_merged_df columns:", list(grassland_merged_df.columns))


Cannot merge: Columns are not the same.
forest_merged_df columns: ['Admin_Unit_Code', 'Sub_Unit_Code', 'Site_Name', 'Plot_Name', 'Location_Type', 'Year', 'Date', 'Start_Time', 'End_Time', 'Observer', 'Visit', 'Interval_Length', 'ID_Method', 'Distance', 'Flyover_Observed', 'Sex', 'Common_Name', 'Scientific_Name', 'AcceptedTSN', 'NPSTaxonCode', 'AOU_Code', 'PIF_Watchlist_Status', 'Regional_Stewardship_Status', 'Temperature', 'Humidity', 'Sky', 'Wind', 'Disturbance', 'Initial_Three_Min_Cnt']
grassland_merged_df columns: ['Admin_Unit_Code', 'Sub_Unit_Code', 'Plot_Name', 'Location_Type', 'Year', 'Date', 'Start_Time', 'End_Time', 'Observer', 'Visit', 'Interval_Length', 'ID_Method', 'Distance', 'Flyover_Observed', 'Sex', 'Common_Name', 'Scientific_Name', 'AcceptedTSN', 'TaxonCode', 'AOU_Code', 'PIF_Watchlist_Status', 'Regional_Stewardship_Status', 'Temperature', 'Humidity', 'Sky', 'Wind', 'Disturbance', 'Previously_Obs', 'Initial_Three_Min_Cnt']


To find which columns are different in forest and grassland dfs:

In [10]:
forest_cols = set(forest_merged_df.columns)
grassland_cols = set(grassland_merged_df.columns)

print("Only in forest_merged_df:", forest_cols - grassland_cols)
print("Only in grassland_merged_df:", grassland_cols - forest_cols)


Only in forest_merged_df: {'Site_Name', 'NPSTaxonCode'}
Only in grassland_merged_df: {'TaxonCode', 'Previously_Obs'}


FOREST DATA CLEANING AND PRE PROCESSING:

Now we find out which columns have nulls in forest df:

In [11]:
null_columns = forest_merged_df.columns[forest_merged_df.isnull().any()].tolist()
print(null_columns)

['Sub_Unit_Code', 'ID_Method', 'Distance', 'Sex', 'AcceptedTSN']


So there are nulls in the above 5 columns. To find the number of nulls in each column:

In [12]:
# List of target columns
columns_to_check = ['Sub_Unit_Code', 'ID_Method', 'Distance', 'Sex', 'AcceptedTSN']

# Count nulls in each column
null_counts = forest_merged_df[columns_to_check].isnull().sum()

print(null_counts)


Sub_Unit_Code    7824
ID_Method           1
Distance           92
Sex              5183
AcceptedTSN         9
dtype: int64


Get the Admin_Unit_Code where the Sub_Unit_Code is null in forest_merged_df:

In [13]:
null_admin_units = forest_merged_df[forest_merged_df['Sub_Unit_Code'].isnull()]['Admin_Unit_Code'].unique()
print("Admin_Unit_Code values where Sub_Unit_Code is null:")
print(null_admin_units)


Admin_Unit_Code values where Sub_Unit_Code is null:
['ANTI' 'CATO' 'CHOH' 'GWMP' 'HAFE' 'MANA' 'MONO' 'PRWI' 'ROCR' 'WOTR']


To get the list of Admin_Unit_Codes where the Sub_Unit_Code is not null in forest df:

In [14]:
# Filter rows where Sub_Unit_Code is NOT null
filtered_df = forest_merged_df[forest_merged_df['Sub_Unit_Code'].notnull()]

# Get the unique pairs of Admin_Unit_Code and Sub_Unit_Code
admin_subunit_pairs = filtered_df[['Admin_Unit_Code', 'Sub_Unit_Code']].drop_duplicates()

# Optional: convert to list of tuples
admin_subunit_list = list(admin_subunit_pairs.itertuples(index=False, name=None))

# Print result
print(admin_subunit_list)


[('GWMP', 'THIS'), ('NACE', 'GREE'), ('NACE', 'PISC'), ('NACE', 'FOWA'), ('NACE', 'OXHI'), ('NACE', 'ANAC'), ('NACE', 'FOCI'), ('NACE', 'FODU')]


Since only Admin_Unit_Code 'NACE' has Sub_Unit_Code and only one row of Admin_Unit_Code GWMP has Sub_Unit_Code we can drop Sub_Unit_Code from the dataframe 

In [15]:
forest_merged_df.drop(columns=['Sub_Unit_Code'], inplace=True)


In [ ]:
forest_merged_df.head()

To print the single row where  'ID_Method' is null:


In [ ]:
rows_with_null_id = forest_merged_df[forest_merged_df['ID_Method'].isnull()]
print(rows_with_null_id)


In [18]:
unique_id_methods = forest_merged_df['ID_Method'].dropna().unique()
print(unique_id_methods)


['Singing' 'Calling' 'Visualization']


To get the 'Common_Name' where the ID_Method is null

In [19]:
# Get Common_Name where ID_Method is null
common_names_with_null_id_method = forest_merged_df.loc[forest_merged_df['ID_Method'].isnull(), 'Common_Name'].unique()

print(common_names_with_null_id_method)


['Red-eyed Vireo']


To fill the ID_Method which is null with the most frequent ID_Method for the same Common_Name

In [ ]:
# Function to fill null ID_Method with mode for each Common_Name group
def fill_id_method_mode(group):
    if group['ID_Method'].isnull().any():
        mode_value = group['ID_Method'].mode()
        if not mode_value.empty:
            group['ID_Method'] = group['ID_Method'].fillna(mode_value[0])
    return group

# Apply the function grouped by Common_Name
forest_merged_df = forest_merged_df.groupby('Common_Name', group_keys=False).apply(fill_id_method_mode)


To verify if the null value is filled with default and confirm:

In [21]:
print(forest_merged_df['ID_Method'].isnull().sum())  # Should print 0


0


To get the unique values of the column 'Sex' in forest_merged_df:

In [22]:
unique_sex_values = forest_merged_df['Sex'].dropna().unique()
print("Unique values in 'Sex':", unique_sex_values)

Unique values in 'Sex': ['Undetermined' 'Male']


Unique sex values already present are 'Undetermined'and 'Male'. So the nulls can be filled with 'Undetermined'

In [23]:
forest_merged_df['Sex'] = forest_merged_df['Sex'].fillna('Undetermined')


To check if nulls are changed:

In [24]:
print(forest_merged_df['Sex'].isnull().sum())   


0


To get the ID_Methods where Distance is null:

In [25]:
id_methods_with_null_distance = forest_merged_df[forest_merged_df['Distance'].isnull()]['ID_Method'].unique()
print("ID_Method values where Distance is null:")
print(id_methods_with_null_distance)


ID_Method values where Distance is null:
['Visualization' 'Singing' 'Calling']


To get the Common_Name of the birds where Distance is null:

In [26]:
print(forest_merged_df[forest_merged_df['Distance'].isnull()][['Common_Name', 'Distance']])


               Common_Name Distance
47           Cedar Waxwing      NaN
50    Red-winged Blackbird      NaN
103           Barn Swallow      NaN
166   Red-winged Blackbird      NaN
322          American Crow      NaN
...                    ...      ...
8215         American Crow      NaN
8239             Fish Crow      NaN
8244         Cedar Waxwing      NaN
8245     Unidentified Crow      NaN
8425         American Crow      NaN

[92 rows x 2 columns]


To get the count of the birds Common_Name wise where Distance is null:

In [27]:
common_name_null_distance_counts = (
    forest_merged_df[forest_merged_df['Distance'].isnull()]
    ['Common_Name']
    .value_counts(dropna=True)
)

print("Count of Common Names with null Distance:")
print(common_name_null_distance_counts)


Count of Common Names with null Distance:
Common_Name
Common Grackle              17
American Crow               13
Cedar Waxwing                8
American Goldfinch           6
Canada Goose                 5
Blue Jay                     5
Great Blue Heron             4
Mourning Dove                4
Turkey Vulture               4
Chimney Swift                3
Unidentified Crow            3
Fish Crow                    3
American Robin               2
Red-winged Blackbird         2
Barn Swallow                 2
Double-crested Cormorant     2
European Starling            2
Red-shouldered Hawk          2
Wood Duck                    1
Mallard                      1
Green Heron                  1
Peregrine Falcon             1
Pileated Woodpecker          1
Name: count, dtype: int64


There are 92 nulls in Distance column. Fill the nulls in Distance with the most frequent Distance for that particular Common_Name of the species in that particular Plot_Name :

In [ ]:
import pandas as pd

# Function to fill null Distance with the mode within each Plot_Name + Common_Name group
def fill_distance_with_mode_by_group(group):
    if group['Distance'].isnull().any():
        mode_val = group['Distance'].mode()
        if not mode_val.empty:
            group['Distance'] = group['Distance'].fillna(mode_val[0])
    return group

# Apply group-wise operation
forest_merged_df = (
    forest_merged_df
    .groupby(['Plot_Name', 'Common_Name'], group_keys=False)
    .apply(fill_distance_with_mode_by_group)
)


To check if the nulls were updated:

In [29]:
print("Remaining nulls in Distance column:", forest_merged_df['Distance'].isnull().sum())

Remaining nulls in Distance column: 57


There are still 57 nulls in Distance column. Fill the nulls in Distance with the most frequent Distance for that particular Common_Name of the species in that particular Admin_Unit :

In [ ]:
import pandas as pd

# Function to fill null Distance with the mode within each Admin_Unit_Code + Common_Name group
def fill_distance_with_mode_by_group(group):
    if group['Distance'].isnull().any():
        mode_val = group['Distance'].mode()
        if not mode_val.empty:
            group['Distance'] = group['Distance'].fillna(mode_val[0])
    return group

# Apply group-wise operation
forest_merged_df = (
    forest_merged_df
    .groupby(['Admin_Unit_Code', 'Common_Name'], group_keys=False)
    .apply(fill_distance_with_mode_by_group)
)

In [31]:
print("Remaining nulls in Distance column:", forest_merged_df['Distance'].isnull().sum())


Remaining nulls in Distance column: 10


There are still 10 rows with nulls in Distance because the most frequent value of Distance for these Common_Name in the particular Admin_Unit_Code is itself null

In [32]:
# Find Common_Name groups where Distance is still null
unfilled_common_names = (
    forest_merged_df[forest_merged_df['Distance'].isnull()]
    ['Common_Name']
    .unique()
)

print("Common_Name values with unfilled nulls in Distance:")
print(unfilled_common_names)


Common_Name values with unfilled nulls in Distance:
['Barn Swallow' 'Peregrine Falcon' 'Turkey Vulture' 'Chimney Swift'
 'Canada Goose' 'Fish Crow' 'Unidentified Crow']


In [33]:
# Filter rows where Distance is still null
null_distance_df = forest_merged_df[forest_merged_df['Distance'].isnull()]

# Group by Common_Name and count how many such rows exist
unfilled_counts = null_distance_df.groupby('Common_Name').size().reset_index(name='Null_Distance_Count')

# Sort descending by count
unfilled_counts = unfilled_counts.sort_values(by='Null_Distance_Count', ascending=False)

print(unfilled_counts)


         Common_Name  Null_Distance_Count
5     Turkey Vulture                    3
0       Barn Swallow                    2
1       Canada Goose                    1
2      Chimney Swift                    1
3          Fish Crow                    1
4   Peregrine Falcon                    1
6  Unidentified Crow                    1


So from above, we see only 10 rows exists, where the Distance is null because there are no rows with no nulls for these types of birds in that particular Admin_Unit. We can assume the most frequent distance of the entire Admin_Unit in these cases and fill with that value. 

In [34]:
import pandas as pd

#Step 3: Find Admin_Unit_Code groups where Distance is still null
still_null = forest_merged_df[forest_merged_df['Distance'].isnull()]

#  For each Admin_Unit_Code with remaining nulls, fill those nulls with mode Distance of entire Admin_Unit_Code
for admin_unit in still_null['Admin_Unit_Code'].unique():
    # Calculate mode of Distance for this Admin_Unit_Code (excluding nulls)
    admin_mode = forest_merged_df.loc[
        (forest_merged_df['Admin_Unit_Code'] == admin_unit) & 
        (forest_merged_df['Distance'].notnull()), 'Distance'
    ].mode()
    
    if not admin_mode.empty:
        # Fill null Distance for this Admin_Unit_Code with its mode
        forest_merged_df.loc[
            (forest_merged_df['Admin_Unit_Code'] == admin_unit) & 
            (forest_merged_df['Distance'].isnull()), 'Distance'
        ] = admin_mode[0]


# Confirm no nulls remain
print("Remaining null Distance values:", forest_merged_df['Distance'].isnull().sum())


Remaining null Distance values: 0


List unique 'AccceptedTSN' values:

In [35]:
unique_accepted_tsn = forest_merged_df['AcceptedTSN'].unique()
print("Unique values in AcceptedTSN:")
print(unique_accepted_tsn)


Unique values in AcceptedTSN:
[179276. 178775. 178195. 179064. 178620. 179124. 554256. 177125. 179443.
 178581. 554383. 178359. 179731. 179150. 554138. 950039. 179801. 178259.
 179435. 179083. 178309. 179853. 178532. 179680. 179236. 179045. 178166.
 179112. 179021. 553526. 178339. 178262. 179009. 179492. 179333. 178448.
 179777. 179759. 178329.     nan 177831. 179104. 179883. 178279. 950033.
 176136. 950029. 178927. 178154. 178979. 179788. 950009. 178844. 178850.
 178541. 950079. 178627. 178625. 178944. 950041. 179737. 175122. 174793.
 178119. 950049. 179023. 175359. 554382. 950010. 174999. 950042. 174773.
 177921. 178991. 950045. 179488. 174717. 179724. 175063. 178443. 178032.
 178001. 178937. 175590. 175272. 178964. 950011. 179637. 179628. 179410.
 175265. 175604. 179034. 179796. 950052. 950036. 950061. 847323. 950031.
 950046. 950035. 179888. 176520. 178277. 950106. 950097. 178186. 175350.]


In [36]:
# Create a mapping from Common_Name to its most frequent non-null AcceptedTSN
accepted_tsn_map = (
    forest_merged_df[forest_merged_df['AcceptedTSN'].notnull()]
    .groupby('Common_Name')['AcceptedTSN']
    .agg(lambda x: x.mode().iloc[0])  # pick the most frequent AcceptedTSN if multiple
    .to_dict()
)

# Function to fill AcceptedTSN from mapping
def fill_accepted_tsn(row):
    if pd.isnull(row['AcceptedTSN']):
        return accepted_tsn_map.get(row['Common_Name'], row['AcceptedTSN'])
    else:
        return row['AcceptedTSN']

# Apply the function row-wise
forest_merged_df['AcceptedTSN'] = forest_merged_df.apply(fill_accepted_tsn, axis=1)


In [37]:
print("Remaining nulls in AcceptedTSN:", forest_merged_df['AcceptedTSN'].isnull().sum())


Remaining nulls in AcceptedTSN: 9


In [38]:
# First, create a mapping from Common_Name to the most frequent non-null AcceptedTSN
accepted_tsn_map = (
    forest_merged_df[forest_merged_df['AcceptedTSN'].notnull()]
    .groupby('Common_Name')['AcceptedTSN']
    .agg(lambda x: x.mode().iloc[0])  # most frequent AcceptedTSN per Common_Name
    .to_dict()
)

# Now, filter rows where AcceptedTSN is null
null_accepted_tsn_rows = forest_merged_df[forest_merged_df['AcceptedTSN'].isnull()]

# For each such row, get the AcceptedTSN from the mapping
null_accepted_tsn_rows = null_accepted_tsn_rows.copy()  # avoid SettingWithCopyWarning
null_accepted_tsn_rows['Filled_AcceptedTSN'] = null_accepted_tsn_rows['Common_Name'].map(accepted_tsn_map)

# Show the results: Common_Name, current null AcceptedTSN, and the mapped AcceptedTSN to fill
print(null_accepted_tsn_rows[['Common_Name', 'AcceptedTSN', 'Filled_AcceptedTSN']])


      Common_Name  AcceptedTSN  Filled_AcceptedTSN
165   House Finch          NaN                 NaN
635   House Finch          NaN                 NaN
638   House Finch          NaN                 NaN
1337  House Finch          NaN                 NaN
3708  House Finch          NaN                 NaN
3912  House Finch          NaN                 NaN
3920  House Finch          NaN                 NaN
3921  House Finch          NaN                 NaN
4034  House Finch          NaN                 NaN


Now we can fill the AcceptedTSN for House Finch which are nulls as 'Unknown':

In [39]:
forest_merged_df['AcceptedTSN'] = forest_merged_df['AcceptedTSN'].fillna('Unknown')


In [40]:
print(forest_merged_df['AcceptedTSN'].isnull().sum())  # Should return 0
print(forest_merged_df['AcceptedTSN'].unique())        # To verify 'Unknown' is included



0
[179276.0 178775.0 178195.0 179064.0 178620.0 179124.0 554256.0 177125.0
 179443.0 178581.0 554383.0 178359.0 179731.0 179150.0 554138.0 950039.0
 179801.0 178259.0 179435.0 179083.0 178309.0 179853.0 178532.0 179680.0
 179236.0 179045.0 178166.0 179112.0 179021.0 553526.0 178339.0 178262.0
 179009.0 179492.0 179333.0 178448.0 179777.0 179759.0 178329.0 'Unknown'
 177831.0 179104.0 179883.0 178279.0 950033.0 176136.0 950029.0 178927.0
 178154.0 178979.0 179788.0 950009.0 178844.0 178850.0 178541.0 950079.0
 178627.0 178625.0 178944.0 950041.0 179737.0 175122.0 174793.0 178119.0
 950049.0 179023.0 175359.0 554382.0 950010.0 174999.0 950042.0 174773.0
 177921.0 178991.0 950045.0 179488.0 174717.0 179724.0 175063.0 178443.0
 178032.0 178001.0 178937.0 175590.0 175272.0 178964.0 950011.0 179637.0
 179628.0 179410.0 175265.0 175604.0 179034.0 179796.0 950052.0 950036.0
 950061.0 847323.0 950031.0 950046.0 950035.0 179888.0 176520.0 178277.0
 950106.0 950097.0 178186.0 175350.0]


In [41]:
# Returns True if any nulls exist, else False
has_nulls = forest_merged_df.isnull().values.any()

if has_nulls:
    print("There are still null values in the DataFrame.")
    # Optional: show which columns have nulls
    print(forest_merged_df.isnull().sum())
else:
    print("✅ The DataFrame has no null values.")


✅ The DataFrame has no null values.


To check if any duplicates in forest_merged_df:

In [42]:
# Count total number of duplicate rows
duplicate_count = forest_merged_df.duplicated().sum()

if duplicate_count > 0:
    print(f"There are {duplicate_count} duplicate rows in the DataFrame.")
else:
    print("✅ No duplicate rows found in the DataFrame.")


There are 4 duplicate rows in the DataFrame.


To print the original + duplicates in forest_merged_df:

In [ ]:
# This will include both the original and its duplicates
duplicates_all = forest_merged_df[forest_merged_df.duplicated(keep=False)]
print(duplicates_all)


Now we can drop the duplicates and keep the first instance :

In [44]:
forest_merged_df = forest_merged_df.drop_duplicates(keep='first')


In [45]:
print(f"Remaining duplicates: {forest_merged_df.duplicated().sum()}")  # Should be 0


Remaining duplicates: 0


Now to check Consisitency of data: For every Common_Name the Scientific_Name should be the same:

In [46]:
# Group by Common_Name and count unique Scientific_Name entries
name_check = forest_merged_df.groupby('Common_Name')['Scientific_Name'].nunique()

# Find where more than one Scientific_Name exists for a Common_Name
inconsistent_names = name_check[name_check > 1]

# Show inconsistent mappings
if not inconsistent_names.empty:
    print("❌ Inconsistent Common_Name ↔ Scientific_Name mappings found:")
    print(inconsistent_names)
else:
    print("✅ Each Common_Name maps to a single Scientific_Name.")


✅ Each Common_Name maps to a single Scientific_Name.


Now to check Consisitency of data: For every Common_Name the AcceptedTSN should be the same:

In [47]:
# Group by Common_Name and count unique AcceptedTSN entries
name_check = forest_merged_df.groupby('Common_Name')['AcceptedTSN'].nunique()

# Find where more than one AcceptedTSN exists for a Common_Name
inconsistent_names = name_check[name_check > 1]

# Show inconsistent mappings
if not inconsistent_names.empty:
    print("❌ Inconsistent Common_Name ↔ AcceptedTSN mappings found:")
    print(inconsistent_names)
else:
    print("✅ Each Common_Name maps to a single AcceptedTSN.")

✅ Each Common_Name maps to a single AcceptedTSN.


Now to check Consisitency of data: For every Common_Name the NPSTaxonCode should be the same:

In [49]:
# Group by Common_Name and count unique NPSTaxonCode entries
name_check = forest_merged_df.groupby('Common_Name')['NPSTaxonCode'].nunique()

# Find where more than one NPSTaxonCode exists for a Common_Name
inconsistent_names = name_check[name_check > 1]

# Show inconsistent mappings
if not inconsistent_names.empty:
    print("❌ Inconsistent Common_Name ↔ NPSTaxonCode mappings found:")
    print(inconsistent_names)
else:
    print("✅ Each Common_Name maps to a single NPSTaxonCode.")

✅ Each Common_Name maps to a single NPSTaxonCode.


Now to check Consisitency of data: For every Common_Name the AOU_Code should be the same:

In [50]:
# Group by Common_Name and count unique AOU_Code entries
name_check = forest_merged_df.groupby('Common_Name')['AOU_Code'].nunique()

# Find where more than one AOU_Code exists for a Common_Name
inconsistent_names = name_check[name_check > 1]

# Show inconsistent mappings
if not inconsistent_names.empty:
    print("❌ Inconsistent Common_Name ↔ AOU_Code mappings found:")
    print(inconsistent_names)
else:
    print("✅ Each Common_Name maps to a single AOU_Code.")

✅ Each Common_Name maps to a single AOU_Code.


Now to check Consisitency of data: For every Common_Name the PIF_Watchlist_Status should be the same:

In [51]:
# Group by Common_Name and count unique PIF_Watchlist_Status entries
name_check = forest_merged_df.groupby('Common_Name')['PIF_Watchlist_Status'].nunique()

# Find where more than one PIF_Watchlist_Status exists for a Common_Name
inconsistent_names = name_check[name_check > 1]

# Show inconsistent mappings
if not inconsistent_names.empty:
    print("❌ Inconsistent Common_Name ↔ PIF_Watchlist_Status mappings found:")
    print(inconsistent_names)
else:
    print("✅ Each Common_Name maps to a single PIF_Watchlist_Status.")

✅ Each Common_Name maps to a single PIF_Watchlist_Status.


Now to check Consisitency of data: For every Common_Name the Regional_Stewardship_Status should be the same:

In [52]:
# Group by Common_Name and count unique Regional_Stewardship_Status entries
name_check = forest_merged_df.groupby('Common_Name')['Regional_Stewardship_Status'].nunique()

# Find where more than one Regional_Stewardship_Status exists for a Common_Name
inconsistent_names = name_check[name_check > 1]

# Show inconsistent mappings
if not inconsistent_names.empty:
    print("❌ Inconsistent Common_Name ↔ Regional_Stewardship_Status mappings found:")
    print(inconsistent_names)
else:
    print("✅ Each Common_Name maps to a single Regional_Stewardship_Status.")

✅ Each Common_Name maps to a single Regional_Stewardship_Status.


Findings: Fron the above we find that for every Common_Name there is a single Scientific_Name, AcceptedTSN, NPSTaxonCode, AOU_Code, PIF_Watchlist_Status, and Regional_Stewardship_Status. So these columns can be combined togoether in a table indexed on the Common_Name.

To check if each unique Plot_Name always maps to the same Admin_Unit_Code and Site_Name in forest_merged_df, we can group by Plot_Name and count unique values in the other two columns and find out:

In [53]:
# Group by Plot_Name and count unique Admin_Unit_Code and Site_Name
consistency_check = forest_merged_df.groupby('Plot_Name')[['Admin_Unit_Code', 'Site_Name']].nunique()

# Find Plot_Names with more than one unique value in either column
inconsistent_plots = consistency_check[(consistency_check['Admin_Unit_Code'] > 1) | 
                                       (consistency_check['Site_Name'] > 1)] 

# Output the inconsistencies
if not inconsistent_plots.empty:
    print("❌ Inconsistencies found for the following Plot_Name(s):")
    print(inconsistent_plots)
else:
    print("✅ Each Plot_Name has consistent Admin_Unit_Code and Site_Name.")


✅ Each Plot_Name has consistent Admin_Unit_Code and Site_Name.


Findings: Fron the above we find that for every Plot_Name there is a single Admin_Unit_Code and Site_Name. So these columns can be combined togoether in a table indexed on the Plot_Name.

Now to analyse the data types of the columns in forest_merged_df:

In [54]:
forest_merged_df.dtypes


Admin_Unit_Code                        object
Site_Name                              object
Plot_Name                              object
Location_Type                          object
Year                                    int64
Date                           datetime64[ns]
Start_Time                             object
End_Time                               object
Observer                               object
Visit                                   int64
Interval_Length                        object
ID_Method                              object
Distance                               object
Flyover_Observed                         bool
Sex                                    object
Common_Name                            object
Scientific_Name                        object
AcceptedTSN                            object
NPSTaxonCode                            int64
AOU_Code                               object
PIF_Watchlist_Status                     bool
Regional_Stewardship_Status       

Now the Start_Time and End_Time are in object format (string) which needs to be converted and only Time should be present :

In [55]:
forest_merged_df['Start_Time'] = pd.to_datetime(forest_merged_df['Start_Time'], format='%H:%M:%S', errors='coerce').dt.time


To confirm if the Start_Time is of correct class datetime.time :

In [56]:
forest_merged_df['Start_Time'].apply(type).unique()


array([<class 'datetime.time'>], dtype=object)

Now check for End_Time and do the same:

In [57]:
forest_merged_df['End_Time'] = pd.to_datetime(forest_merged_df['End_Time'], format='%H:%M:%S', errors='coerce').dt.time

In [58]:
forest_merged_df['Start_Time'].apply(type).unique()


array([<class 'datetime.time'>], dtype=object)

GRASSLAND DATA CLEANING AND PRE PROCESSING:

To list down the columns with nulls in Grassland df:

In [59]:
null_columns = grassland_merged_df.columns[grassland_merged_df.isnull().any()].tolist()
print(null_columns)


['Sub_Unit_Code', 'ID_Method', 'Distance', 'AcceptedTSN', 'TaxonCode']


In [60]:
# List of target columns
columns_to_check = ['Sub_Unit_Code', 'ID_Method', 'Distance', 'AcceptedTSN', 'TaxonCode']

# Count nulls in each column
null_counts = grassland_merged_df[columns_to_check].isnull().sum()

print(null_counts)

Sub_Unit_Code    8531
ID_Method           1
Distance         1394
AcceptedTSN        24
TaxonCode           2
dtype: int64


In [61]:
grassland_merged_df['Sub_Unit_Code'].dropna().unique()


array([], dtype=float64)

The Sub_Unit_Code is entire column with nulls in grassland_merged_df and the column can be dropped

In [62]:
grassland_merged_df.drop(columns=['Sub_Unit_Code'], inplace=True)

In [ ]:
grassland_merged_df.head()

To print the single row where  'ID_Method' is null:

In [ ]:
rows_with_null_id = grassland_merged_df[grassland_merged_df['ID_Method'].isnull()]
print(rows_with_null_id)

In [65]:
unique_id_methods = grassland_merged_df['ID_Method'].dropna().unique()
print(unique_id_methods)

['Singing' 'Calling' 'Visualization']


Get Common_Name where ID_Method is null:

In [66]:
common_names_with_null_id_method = grassland_merged_df.loc[grassland_merged_df['ID_Method'].isnull(), 'Common_Name'].unique()
print(common_names_with_null_id_method)


['American Crow']


To fill the ID_Method which is null with the most frequent ID_Method for the same Common_Name:

In [ ]:
# Function to fill null ID_Method with mode for each Common_Name group
def fill_id_method_mode(group):
    if group['ID_Method'].isnull().any():
        mode_value = group['ID_Method'].mode()
        if not mode_value.empty:
            group['ID_Method'] = group['ID_Method'].fillna(mode_value[0])
    return group

# Apply the function grouped by Common_Name
grassland_merged_df = grassland_merged_df.groupby('Common_Name', group_keys=False).apply(fill_id_method_mode)

To verify if the null value is filled and confirm:

In [68]:
print(grassland_merged_df['ID_Method'].isnull().sum())  # Should print 0

0


To get the Distance which are null Common_Name wise:

In [69]:
common_name_null_distance_counts = (
    grassland_merged_df[grassland_merged_df['Distance'].isnull()]
    ['Common_Name']
    .value_counts(dropna=True)
)
print("Count of Common Names with null Distance:")
print(common_name_null_distance_counts)


Count of Common Names with null Distance:
Common_Name
European Starling                321
Cedar Waxwing                    255
Common Grackle                   127
Barn Swallow                     127
American Goldfinch               101
Red-winged Blackbird              83
Mourning Dove                     52
Turkey Vulture                    44
Brown-headed Cowbird              34
American Crow                     32
Tree Swallow                      29
Blue Jay                          20
American Robin                    20
Northern Rough-winged Swallow     16
Unidentified Crow                 14
Canada Goose                      14
Great Blue Heron                  12
Unidentified Swallow              11
Rock Dove                         11
Fish Crow                         10
Killdeer                           7
Northern Cardinal                  6
Bald Eagle                         5
House Finch                        5
American Kestrel                   5
Chimney Swift        

There are 1394 nulls in Distance Column. Fill the nulls in Distance with the most frequent Distance for that particular Common_Name of the species in that particular Plot_Name :

In [ ]:
import pandas as pd

# Function to fill null Distance with the mode within each Plot_Name + Common_Name group
def fill_distance_with_mode_by_group(group):
    if group['Distance'].isnull().any():
        mode_val = group['Distance'].mode()
        if not mode_val.empty:
            group['Distance'] = group['Distance'].fillna(mode_val[0])
    return group

# Apply group-wise operation
grassland_merged_df = (
    grassland_merged_df
    .groupby(['Plot_Name', 'Common_Name'], group_keys=False)
    .apply(fill_distance_with_mode_by_group)
)


To check if the nulls were updated:

In [71]:
print("Remaining nulls in Distance column:", grassland_merged_df['Distance'].isnull().sum())

Remaining nulls in Distance column: 748


There are still 748 rows with nulls in Distance because the most frequent value of Distance for these Common_Name in the particular Plot_Name is itself null. Now fill the nulls in Distance with the most frequent Distance for that particular Common_Name of the species in that particular Admin_Unit :

In [ ]:
import pandas as pd

# Function to fill null Distance with the mode within each Admin_Unit_Code + Common_Name group
def fill_distance_with_mode_by_group(group):
    if group['Distance'].isnull().any():
        mode_val = group['Distance'].mode()
        if not mode_val.empty:
            group['Distance'] = group['Distance'].fillna(mode_val[0])
    return group

# Apply group-wise operation
grassland_merged_df = (
    grassland_merged_df
    .groupby(['Admin_Unit_Code', 'Common_Name'], group_keys=False)
    .apply(fill_distance_with_mode_by_group)
)


To check if the nulls were updated:

In [78]:
print("Remaining nulls in Distance column:", grassland_merged_df['Distance'].isnull().sum())

Remaining nulls in Distance column: 49


There are still 49 rows with nulls in Distance because the most frequent value of Distance for these Common_Name in the particular Admin_Unit_Code is itself null

In [73]:
# Find Common_Name groups where Distance is still null
unfilled_common_names = (
    grassland_merged_df[grassland_merged_df['Distance'].isnull()]
    ['Common_Name']
    .unique()
)

print("Common_Name values with unfilled nulls in Distance:")
print(unfilled_common_names)



Common_Name values with unfilled nulls in Distance:
['Bald Eagle' 'Wood Duck' 'Unidentified Crow' 'Canada Goose'
 'Great Blue Heron' 'Purple Martin' 'Unidentified Swallow' 'Black Vulture'
 'American Kestrel']


In [74]:
# Filter rows where Distance is still null
null_distance_df = grassland_merged_df[grassland_merged_df['Distance'].isnull()]

# Group by Common_Name and count how many such rows exist
unfilled_counts = null_distance_df.groupby('Common_Name').size().reset_index(name='Null_Distance_Count')

# Sort descending by count
unfilled_counts = unfilled_counts.sort_values(by='Null_Distance_Count', ascending=False)

print(unfilled_counts)


            Common_Name  Null_Distance_Count
4      Great Blue Heron                   12
7  Unidentified Swallow                   11
6     Unidentified Crow                    7
3          Canada Goose                    6
1            Bald Eagle                    5
0      American Kestrel                    3
5         Purple Martin                    3
2         Black Vulture                    1
8             Wood Duck                    1


So from above, we see only 49 rows exists, where the Distance is null because there are no rows with no nulls for these types of birds in that particular Admin_Unit. 
We can assume the most frequent distance of the entire Admin_Unit in these cases and fill with that value.


In [75]:
import pandas as pd

#Find Admin_Unit_Code groups where Distance is still null
still_null = grassland_merged_df[grassland_merged_df['Distance'].isnull()]

#  For each Admin_Unit_Code with remaining nulls, fill those nulls with mode Distance of entire Admin_Unit_Code
for admin_unit in still_null['Admin_Unit_Code'].unique():
    # Calculate mode of Distance for this Admin_Unit_Code (excluding nulls)
    admin_mode = grassland_merged_df.loc[
        (grassland_merged_df['Admin_Unit_Code'] == admin_unit) & 
        (grassland_merged_df['Distance'].notnull()), 'Distance'
    ].mode()
    
    if not admin_mode.empty:
        # Fill null Distance for this Admin_Unit_Code with its mode
        grassland_merged_df.loc[
            (grassland_merged_df['Admin_Unit_Code'] == admin_unit) & 
            (grassland_merged_df['Distance'].isnull()), 'Distance'
        ] = admin_mode[0]



 Confirm no nulls remain in Distance in grassland df:

In [76]:
print("Remaining null Distance values:", grassland_merged_df['Distance'].isnull().sum())

Remaining null Distance values: 0


List unique 'AccceptedTSN' values:

In [77]:
unique_accepted_tsn = grassland_merged_df['AcceptedTSN'].unique()
print("Unique values in AcceptedTSN:")
print(unique_accepted_tsn)

Unique values in AcceptedTSN:
[179435. 179801. 179333. 179443. 179731. 178279. 179150. 179124. 177125.
 179276. 179492. 178431. 178581. 950041. 179112. 179083. 179737. 178309.
 179759. 175265. 178195. 554138. 174999. 179680. 554383. 178625. 179777.
 179724. 554256. 178359. 179064. 178532. 178944. 179853. 178339. 178620.
 179104. 175420. 179045. 179034. 178448. 179366. 179236. 178627. 178329.
 553526. 178541. 178262. 179023. 178001. 175272. 179637.     nan 178154.
 179021. 175350. 176520. 175622. 179883. 950033. 178775. 178979. 179145.
 177831. 178259. 950045. 175122. 178166. 179628. 175359. 950052. 178964.
 175309. 178032. 177071. 179736. 178443. 179009. 179796. 174773. 176136.
 179888. 178991. 178341. 178927. 950035. 179788. 178464. 179314. 178423.
 174861. 950040. 179032. 950039. 178277. 178937. 178119. 178842. 950010.
 950061. 178844. 179462. 177921. 179488. 950036. 950031. 178344.]


Check if AcceptedTSN value exists in any other row for the same Common_Name:

In [78]:
# Step 1: Get Common_Name values where AcceptedTSN is null
common_names_with_null_acceptedtsn = grassland_merged_df.loc[grassland_merged_df['AcceptedTSN'].isna(), 'Common_Name'].unique()

# Step 2: Filter rows with those Common_Names and where AcceptedTSN is NOT null
rows_with_taxoncode = grassland_merged_df[
    (grassland_merged_df['Common_Name'].isin(common_names_with_null_acceptedtsn)) &
    (grassland_merged_df['AcceptedTSN'].notna())
]

# Step 3: See if any AcceptedTSN exists for those Common_Names
print(rows_with_taxoncode[['Common_Name', 'AcceptedTSN']].drop_duplicates())


Empty DataFrame
Columns: [Common_Name, AcceptedTSN]
Index: []


To fill all nulls in AcceptedTSN with 'Unknown'

In [79]:
grassland_merged_df['AcceptedTSN'] = grassland_merged_df['AcceptedTSN'].fillna('Unknown')

To check TaxonCode for nulls: List unique TaxonCode

In [81]:
unique_taxoncode = grassland_merged_df['TaxonCode'].unique()
print("Unique values in TaxonCode:")
print(unique_taxoncode)

Unique values in TaxonCode:
[ 84781.  87184.  83867.  84790.  87106.  85757.  94257.  94228.  82737.
  83803.  85643.  87391.  88350. 773819.  94215.  93655.  87112.  85791.
  87136.  79468.  84865. 266957.  76625.  86252. 265876.  89977.  87156.
  87098. 264079.  85846.  93634.  88296.  92708.  88038.  85824.  88394.
  94206.  80444.  93613.  93601.  87409.  84704.  95286.  89979.  85813.
 263793.  88306.  85739.  93589.  83046.  79476.  86204. 926917.  84820.
  93587.  79563.  77646.  82543.  88071. 890949.  90935.  92746.  94251.
  89102.  84936. 890952.  77836.  84833.  86194.  79572. 890945.  92730.
  79517.  83884.  82677.     nan  87404.  93573.  87177.  84416.  86451.
  88076.  93552.  85827.  92689. 774021.  87168.  87427.  83846.  87382.
 773820.  93599. 773778.  85755.  92700.  83980.  91010. 773765. 890946.
  91012.  85610.  82956.  85638. 890943. 773818.  85830.]


In [82]:
# Filter rows where TaxonCode is null and get the corresponding Common_Name
null_taxoncode_common_names = grassland_merged_df.loc[grassland_merged_df['TaxonCode'].isna(), 'Common_Name']
print(null_taxoncode_common_names)


2596     Northwestern Crow
5556    Chinese Pond-Heron
Name: Common_Name, dtype: object


In [83]:
# Step 1: Get Common_Name values where TaxonCode is null
common_names_with_null_taxoncode = grassland_merged_df.loc[grassland_merged_df['TaxonCode'].isna(), 'Common_Name'].unique()

# Step 2: Filter rows with those Common_Names and where TaxonCode is NOT null
rows_with_taxoncode = grassland_merged_df[
    (grassland_merged_df['Common_Name'].isin(common_names_with_null_taxoncode)) &
    (grassland_merged_df['TaxonCode'].notna())
]

# Step 3: See if any TaxonCode exists for those Common_Names
print(rows_with_taxoncode[['Common_Name', 'TaxonCode']].drop_duplicates())


Empty DataFrame
Columns: [Common_Name, TaxonCode]
Index: []


Fill TaxonCode with nulls as 'Unknown' in grassland_merged_df:

In [84]:
grassland_merged_df['TaxonCode'] = grassland_merged_df['TaxonCode'].fillna('Unknown')


In [85]:
# Returns True if any nulls exist, else False
has_nulls = grassland_merged_df.isnull().values.any()

if has_nulls:
    print("There are still null values in the DataFrame.")
    # Optional: show which columns have nulls
    print(grassland_merged_df.isnull().sum())
else:
    print("✅ The DataFrame has no null values.")

✅ The DataFrame has no null values.


To check if any duplicates in grassland_merged_df:

In [86]:
# Count total number of duplicate rows
duplicate_count = grassland_merged_df.duplicated().sum()

if duplicate_count > 0:
    print(f"There are {duplicate_count} duplicate rows in the DataFrame.")
else:
    print("✅ No duplicate rows found in the DataFrame.")


There are 1705 duplicate rows in the DataFrame.


To print the original + duplicates in grassland_merged_df:

In [ ]:
# This will include both the original and its duplicates
duplicates_all = grassland_merged_df[grassland_merged_df.duplicated(keep=False)]
print(duplicates_all)


Now we can drop the duplicates and keep the first instance :

In [88]:
grassland_merged_df = grassland_merged_df.drop_duplicates(keep='first')

In [89]:
print(f"Remaining duplicates: {grassland_merged_df.duplicated().sum()}")  # Should be 0


Remaining duplicates: 0


Now to analyse the data types of the columns in grassland_merged_df:

In [90]:
grassland_merged_df.dtypes

Admin_Unit_Code                        object
Plot_Name                              object
Location_Type                          object
Year                                    int64
Date                           datetime64[ns]
Start_Time                             object
End_Time                               object
Observer                               object
Visit                                   int64
Interval_Length                        object
ID_Method                              object
Distance                               object
Flyover_Observed                         bool
Sex                                    object
Common_Name                            object
Scientific_Name                        object
AcceptedTSN                            object
TaxonCode                              object
AOU_Code                               object
PIF_Watchlist_Status                     bool
Regional_Stewardship_Status              bool
Temperature                       

Now the Start_Time and End_Time are in object format (string) which needs to be converted and only Time should be present :

In [91]:
grassland_merged_df['Start_Time'] = pd.to_datetime(grassland_merged_df['Start_Time'], format='%H:%M:%S', errors='coerce').dt.time


To confirm if the Start_Time is of correct class datetime.time :

In [92]:
grassland_merged_df['Start_Time'].apply(type).unique()

array([<class 'datetime.time'>], dtype=object)

Now check for End_Time and do the same:

In [93]:
forest_merged_df['End_Time'] = pd.to_datetime(forest_merged_df['End_Time'], format='%H:%M:%S', errors='coerce').dt.time
forest_merged_df['Start_Time'].apply(type).unique()

array([<class 'datetime.time'>], dtype=object)

In [94]:
only_false = grassland_merged_df['Previously_Obs'].eq(False).all()
print("All values are False:" if only_false else "There are True or missing values.")


All values are False:


In [95]:
print(grassland_merged_df['Previously_Obs'].unique())


[False]


We can drop the column 'Previously_Obs' as all values are 'False'

In [96]:
grassland_merged_df.drop(columns=['Previously_Obs'], inplace=True)


In [97]:
print(grassland_merged_df.columns)


Index(['Admin_Unit_Code', 'Plot_Name', 'Location_Type', 'Year', 'Date',
       'Start_Time', 'End_Time', 'Observer', 'Visit', 'Interval_Length',
       'ID_Method', 'Distance', 'Flyover_Observed', 'Sex', 'Common_Name',
       'Scientific_Name', 'AcceptedTSN', 'TaxonCode', 'AOU_Code',
       'PIF_Watchlist_Status', 'Regional_Stewardship_Status', 'Temperature',
       'Humidity', 'Sky', 'Wind', 'Disturbance', 'Initial_Three_Min_Cnt'],
      dtype='object')


In [98]:
print(forest_merged_df.columns)

Index(['Admin_Unit_Code', 'Site_Name', 'Plot_Name', 'Location_Type', 'Year',
       'Date', 'Start_Time', 'End_Time', 'Observer', 'Visit',
       'Interval_Length', 'ID_Method', 'Distance', 'Flyover_Observed', 'Sex',
       'Common_Name', 'Scientific_Name', 'AcceptedTSN', 'NPSTaxonCode',
       'AOU_Code', 'PIF_Watchlist_Status', 'Regional_Stewardship_Status',
       'Temperature', 'Humidity', 'Sky', 'Wind', 'Disturbance',
       'Initial_Three_Min_Cnt'],
      dtype='object')


Rename NPSTaxonCode in forest_merged_df as TaxonCode :

In [99]:
forest_merged_df.rename(columns={'NPSTaxonCode': 'TaxonCode'}, inplace=True)


Now creat df grassland_plot_details and forest_plot_details which stores the unique set of columns 'Admin_Unit_Code', 'Location_Type' for a particular plot_name. In case of forst_plot_details column 'Site_Name' is also added. Concatenate both to get one plot_details df

In [100]:
import pandas as pd

# Select relevant columns from both DataFrames, adding 'Site_Name' only where available
grassland_plot_details = grassland_merged_df[['Plot_Name', 'Admin_Unit_Code', 'Location_Type']].copy()
grassland_plot_details['Site_Name'] = pd.NA  # Add missing column with nulls

forest_plot_details = forest_merged_df[['Plot_Name', 'Admin_Unit_Code', 'Location_Type', 'Site_Name']].copy()

# Concatenate the two
plot_details = pd.concat([grassland_plot_details, forest_plot_details], ignore_index=True)

# Drop duplicates to ensure one row per unique Plot_Name (keeping first occurrence)
plot_details = plot_details.drop_duplicates(subset=['Plot_Name'])

# Set 'Plot_Name' as index
plot_details.set_index('Plot_Name', inplace=True)

# Final Plot_Details table
Plot_Details = plot_details

# Display result (optional)
print(Plot_Details.head())



          Admin_Unit_Code Location_Type Site_Name
Plot_Name                                        
ANTI-0054            ANTI     Grassland       NaN
ANTI-0028            ANTI     Grassland       NaN
ANTI-0027            ANTI     Grassland       NaN
ANTI-0018            ANTI     Grassland       NaN
ANTI-0105            ANTI     Grassland       NaN


In [101]:
print(plot_details.shape)

(609, 3)


Create species_details df with the columns 'Common_Name', 'Scientific_Name', 'AcceptedTSN', 'TaxonCode',
    'AOU_Code', 'PIF_Watchlist_Status', 'Regional_Stewardship_Status' as all these values are unique for a particular Common Name. Also the index can be the Common_Name

In [ ]:
# Define the columns we need for Species_Details
species_columns = [
    'Common_Name', 'Scientific_Name', 'AcceptedTSN', 'TaxonCode',
    'AOU_Code', 'PIF_Watchlist_Status', 'Regional_Stewardship_Status'
]

# Extract relevant columns from both dataframes
grassland_species = grassland_merged_df[species_columns].copy()
forest_species = forest_merged_df[species_columns].copy()

# Concatenate both species DataFrames
species_combined = pd.concat([grassland_species, forest_species], ignore_index=True)

# Drop duplicates to ensure unique rows per Common_Name
species_details = species_combined.drop_duplicates(subset='Common_Name')

# Set Common_Name as the index
species_details.set_index('Common_Name', inplace=True)

# Display the resulting Species_Details table
print(species_details.head())


Now we create 2 dfs : forest_bird_observations and grassland_bird_observations with 'Plot_Name', 'Date', 'Start_Time', 'End_Time', 'Observer','Visit', 'Interval_Length', 'ID_Method', 'Distance','Flyover_Observed', 'Sex', 'Common_Name', 'Temperature', 'Humidity', 'Sky', 'Wind','Disturbance', 'Initial_Three_Min_Cnt'columns

In [103]:
# Define the required columns
observation_columns = [
    'Plot_Name', 'Date', 'Start_Time', 'End_Time', 'Observer', 'Visit',
    'Interval_Length', 'ID_Method', 'Distance', 'Flyover_Observed', 'Sex',
    'Common_Name', 'Temperature', 'Humidity', 'Sky', 'Wind',
    'Disturbance', 'Initial_Three_Min_Cnt'
]

# Create forest bird observations DataFrame
forest_bird_observations = forest_merged_df[observation_columns].copy()

# Create grassland bird observations DataFrame
grassland_bird_observations = grassland_merged_df[observation_columns].copy()




Now need to move data from 4 dfs namely: forest_bird_observations, grassland_bird_observations, Plot_details and species_details into 4 tables in SQL Workbench . The Plot_Name will be the primary key in Plot_details table and 'Common_Name' will be the primary key in species_details table and these 2 columns will be the foreign keys in forest_bird_observations and grassland_bird_observation.

pip install mysql-connector-python sqlalchemy


Create lookup tables to avoid long strings in columns 'Sky', 'Wind','Disturbance' :

In [107]:
def create_combined_lookup_and_merge(df1, df2, column_name):
    # Get unique non-null values from both DataFrames
    combined_unique_values = pd.Series(
        pd.concat([df1[column_name], df2[column_name]])
        .dropna()
        .unique()
    ).sort_values().reset_index(drop=True)
    
    # Create a lookup table
    lookup_df = pd.DataFrame({
        f'{column_name}_Code': range(1, len(combined_unique_values) + 1),
        column_name: combined_unique_values
    })

    # Merge the lookup into both DataFrames and drop original column
    df1 = df1.merge(lookup_df, on=column_name, how='left').drop(columns=[column_name])
    df2 = df2.merge(lookup_df, on=column_name, how='left').drop(columns=[column_name])
    
    return df1, df2, lookup_df



In [ ]:
# Sky
forest_bird_observations, grassland_bird_observations, sky_lookup = create_combined_lookup_and_merge(
    forest_bird_observations, grassland_bird_observations, 'Sky')

# Wind
forest_bird_observations, grassland_bird_observations, wind_lookup = create_combined_lookup_and_merge(
    forest_bird_observations, grassland_bird_observations, 'Wind')

# Disturbance
forest_bird_observations, grassland_bird_observations, disturbance_lookup = create_combined_lookup_and_merge(
    forest_bird_observations, grassland_bird_observations, 'Disturbance')



In [109]:
print("Forest columns:", forest_bird_observations.columns)
print("Grassland columns:", grassland_bird_observations.columns)


Forest columns: Index(['Plot_Name', 'Date', 'Start_Time', 'End_Time', 'Observer', 'Visit',
       'Interval_Length', 'ID_Method', 'Distance', 'Flyover_Observed', 'Sex',
       'Common_Name', 'Temperature', 'Humidity', 'Initial_Three_Min_Cnt',
       'Sky_Code', 'Wind_Code', 'Disturbance_Code'],
      dtype='object')
Grassland columns: Index(['Plot_Name', 'Date', 'Start_Time', 'End_Time', 'Observer', 'Visit',
       'Interval_Length', 'ID_Method', 'Distance', 'Flyover_Observed', 'Sex',
       'Common_Name', 'Temperature', 'Humidity', 'Initial_Three_Min_Cnt',
       'Sky_Code', 'Wind_Code', 'Disturbance_Code'],
      dtype='object')


In [ ]:
print (forest_bird_observations.head())

In [ ]:
print (grassland_bird_observations.head())

To check and clean Interval_Length column:

In [113]:
all_end_with_min = forest_bird_observations['Interval_Length'].astype(str).str.endswith('min').all()
print("All values end with 'min':", all_end_with_min)


All values end with 'min': True


In [115]:
print(forest_bird_observations['Interval_Length'].unique())


['0-2.5 min' '2.5 - 5 min' '5 - 7.5 min' '7.5 - 10 min']


In [116]:
print(grassland_bird_observations['Interval_Length'].unique())

['0-2.5 min' '7.5 - 10 min' '2.5 - 5 min' '5 - 7.5 min']


 Add Interval_Start, Interval_End, and Interval_Avg to forest_bird_observations and grassland_bird_observations

In [117]:
# Clean and extract interval values
interval_split = forest_bird_observations['Interval_Length'].astype(str).str.extract(
    r'(?P<Interval_Start>[\d\.]+)\s*-\s*(?P<Interval_End>[\d\.]+)'
)

# Convert to float
interval_split['Interval_Start'] = interval_split['Interval_Start'].astype(float)
interval_split['Interval_End'] = interval_split['Interval_End'].astype(float)

# Calculate average
interval_split['Interval_Avg'] = (interval_split['Interval_Start'] + interval_split['Interval_End']) / 2

# Merge into the original DataFrame
forest_bird_observations = pd.concat([forest_bird_observations, interval_split], axis=1)


In [118]:
# Clean and extract interval values
interval_split = grassland_bird_observations['Interval_Length'].astype(str).str.extract(
    r'(?P<Interval_Start>[\d\.]+)\s*-\s*(?P<Interval_End>[\d\.]+)'
)

# Convert to float
interval_split['Interval_Start'] = interval_split['Interval_Start'].astype(float)
interval_split['Interval_End'] = interval_split['Interval_End'].astype(float)

# Calculate average
interval_split['Interval_Avg'] = (interval_split['Interval_Start'] + interval_split['Interval_End']) / 2

# Merge into the original DataFrame
grassland_bird_observations = pd.concat([grassland_bird_observations, interval_split], axis=1)


In [ ]:
print(grassland_bird_observations.head())

Distance can also be cleaned and processed. First we extract unique distance categories from both dataframes

In [121]:
def get_unique_distance_categories(*dfs):
    unique_distances = set()
    for df in dfs:
        unique_distances.update(df['Distance'].dropna().unique())
    return sorted(unique_distances)


Then create a lookup table with the parsed information:

In [123]:
import pandas as pd
import numpy as np

def parse_distance(distance_str):
    if pd.isnull(distance_str):
        return (np.nan, np.nan, 'Unknown')
    
    s = distance_str.strip()
    
    if "<=" in s:
        val = float(s.replace("<= ", "").replace("Meters", "").strip())
        label = f"<= {int(val)}m"
        return (0, val, label)
    elif " - " in s:
        parts = s.replace("Meters", "").strip().split(" - ")
        start = float(parts[0])
        end = float(parts[1])
        label = f"{int(start)}-{int(end)}m"
        return (start, end, label)
    elif ">=" in s:
        val = float(s.replace(">= ", "").replace("Meters", "").strip())
        label = f">= {int(val)}m"
        return (val, np.nan, label)
    elif ">" in s:
        val = float(s.replace("> ", "").replace("Meters", "").strip())
        label = f"> {int(val)}m"
        return (val, np.nan, label)
    else:
        return (np.nan, np.nan, 'Other')

# Get unique categories from both dfs
unique_distances = get_unique_distance_categories(forest_bird_observations, grassland_bird_observations)

# Parse and build lookup data
lookup_data = [parse_distance(d) for d in unique_distances]

# Create lookup DataFrame
distance_lookup_df = pd.DataFrame(lookup_data, columns=['Distance_Start', 'Distance_End', 'Distance_Category'])
distance_lookup_df['Distance_Raw'] = unique_distances  # original strings for reference

# Assign a numeric code as primary key
distance_lookup_df.reset_index(inplace=True)
distance_lookup_df.rename(columns={'index': 'Distance_Code'}, inplace=True)

distance_lookup_df


,Distance_Code,Distance_Start,Distance_End,Distance_Category,Distance_Raw
0,0,50.0,100.0,50-100m,50 - 100 Meters
1,1,0.0,50.0,<= 50m,<= 50 Meters


Map Distance strings in bird observations to this lookup

In [124]:
# Map Distance to Distance_Code using the raw string
mapping_dict = dict(zip(distance_lookup_df['Distance_Raw'], distance_lookup_df['Distance_Code']))

# Apply to both dataframes
forest_bird_observations['Distance_Code'] = forest_bird_observations['Distance'].map(mapping_dict)
grassland_bird_observations['Distance_Code'] = grassland_bird_observations['Distance'].map(mapping_dict)


Connection to SQL:

In [127]:
from sqlalchemy import create_engine

# Define your MySQL connection string
username = 'root'
password = 'Raji'
host = 'localhost'
port = '3306'
database = 'birdobservation_db'

# Create connection engine
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}')

Uploading Lookup Tables First:

In [ ]:
plot_details.to_sql('Plot_details', con=engine, if_exists='append', index=False)


In [ ]:
print(plot_details['Plot_Name'].isnull().sum())
print(plot_details[plot_details['Plot_Name'].isnull()])


In [133]:
plot_details.head()

,Admin_Unit_Code,Location_Type,Site_Name
Plot_Name,,,
ANTI-0054,ANTI,Grassland,NaN
ANTI-0028,ANTI,Grassland,NaN
ANTI-0027,ANTI,Grassland,NaN
ANTI-0018,ANTI,Grassland,NaN
ANTI-0105,ANTI,Grassland,NaN


In [134]:
plot_details = plot_details.reset_index()


In [ ]:
plot_details.to_sql('Plot_details', con=engine, if_exists='append', index=False)


In [140]:
species_details.reset_index()

,Common_Name,Scientific_Name,AcceptedTSN,TaxonCode,AOU_Code,PIF_Watchlist_Status,Regional_Stewardship_Status
0,Chipping Sparrow,Spizella passerina,179435.0,84781.0,CHSP,False,False
1,Eastern Bluebird,Sialia sialis,179801.0,87184.0,EABL,False,False
2,Grasshopper Sparrow,Ammodramus savannarum,179333.0,83867.0,GRSP,False,False
3,Field Sparrow,Spizella pusilla,179443.0,84790.0,FISP,False,True
4,American Crow,Corvus brachyrhynchos,179731.0,87106.0,AMCR,False,False
...,...,...,...,...,...,...,...
121,Unidentified Woodpecker,Picadae,847323.0,816311,UNWO,False,False
122,Yellow-rumped Warbler,Setophaga coronata,950046.0,773779,YRWA,False,False
123,Nashville Warbler,Oreothlypis ruficapilla,950106.0,773768,NAWA,False,False
124,Tennessee Warbler,Oreothlypis peregrina,950097.0,773770,TEWA,False,False


In [ ]:
species_details.to_sql('Species_details', con=engine, if_exists='append', index=False)

In [142]:
species_details = species_details.reset_index()


In [ ]:
species_details.to_sql('Species_details', con=engine, if_exists='append', index=False)


In [150]:
sky_lookup = sky_lookup.reset_index()

In [151]:
print(sky_lookup.columns)


Index(['level_0', 'index', 'Sky_Code', 'Sky'], dtype='object')


In [152]:
sky_lookup_cleaned = sky_lookup[['Sky_Code', 'Sky']].rename(columns={'Sky': 'Sky_Description'})


In [ ]:
sky_lookup_cleaned.to_sql('Sky_Lookup', con=engine, if_exists='append', index=False)


In [ ]:
# For wind
wind_lookup_cleaned = wind_lookup[['Wind_Code', 'Wind']].rename(columns={'Wind': 'Wind_Description'})
wind_lookup_cleaned.to_sql('Wind_Lookup', con=engine, if_exists='append', index=False)

# For disturbance
disturbance_lookup_cleaned = disturbance_lookup[['Disturbance_Code', 'Disturbance']].rename(columns={'Disturbance': 'Disturbance_Description'})
disturbance_lookup_cleaned.to_sql('Disturbance_Lookup', con=engine, if_exists='append', index=False)


In [155]:
print(distance_lookup_df.columns)


Index(['Distance_Code', 'Distance_Start', 'Distance_End', 'Distance_Category',
       'Distance_Raw'],
      dtype='object')


In [156]:
distance_lookup_cleaned = distance_lookup_df[[
    'Distance_Code', 'Distance_Raw', 'Distance_Start', 'Distance_End', 'Distance_Category'
]]


In [157]:
distance_lookup_cleaned.to_sql('Distance_Lookup', con=engine, if_exists='append', index=False)


C:\Users\DELL\AppData\Local\Temp\ipykernel_1332\1366051435.py:1: UserWarning: The provided table name 'Distance_Lookup' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  distance_lookup_cleaned.to_sql('Distance_Lookup', con=engine, if_exists='append', index=False)


2

In [158]:
print(grassland_bird_observations.columns)
print(forest_bird_observations.columns)


Index(['Plot_Name', 'Date', 'Start_Time', 'End_Time', 'Observer', 'Visit',
       'Interval_Length', 'ID_Method', 'Distance', 'Flyover_Observed', 'Sex',
       'Common_Name', 'Temperature', 'Humidity', 'Initial_Three_Min_Cnt',
       'Sky_Code', 'Wind_Code', 'Disturbance_Code', 'Interval_Start',
       'Interval_End', 'Interval_Avg', 'Distance_Code'],
      dtype='object')
Index(['Plot_Name', 'Date', 'Start_Time', 'End_Time', 'Observer', 'Visit',
       'Interval_Length', 'ID_Method', 'Distance', 'Flyover_Observed', 'Sex',
       'Common_Name', 'Temperature', 'Humidity', 'Initial_Three_Min_Cnt',
       'Sky_Code', 'Wind_Code', 'Disturbance_Code', 'Interval_Start',
       'Interval_End', 'Interval_Avg', 'Distance_Code'],
      dtype='object')


In [159]:
grassland_bird_observations = grassland_bird_observations.drop(columns=['index', 'level_0', 'Observation_ID'], errors='ignore')
forest_bird_observations = forest_bird_observations.drop(columns=['index', 'level_0', 'Observation_ID'], errors='ignore')


In [ ]:
print(grassland_bird_observations.columns)
print(forest_bird_observations.columns)

In [163]:
print(grassland_bird_observations.dtypes)

Plot_Name                 object
Date                      object
Start_Time                object
End_Time                  object
Observer                  object
Visit                      int64
Interval_Length           object
ID_Method                 object
Distance                  object
Flyover_Observed            bool
Sex                       object
Common_Name               object
Temperature              float64
Humidity                 float64
Initial_Three_Min_Cnt       bool
Sky_Code                   int64
Wind_Code                  int64
Disturbance_Code           int64
Interval_Start           float64
Interval_End             float64
Interval_Avg             float64
Distance_Code              int64
dtype: object


In [ ]:
grassland_bird_observations['Date'] = pd.to_datetime(grassland_bird_observations['Date']).dt.date
grassland_bird_observations['Start_Time'] = pd.to_datetime(grassland_bird_observations['Start_Time']).dt.time
grassland_bird_observations['End_Time'] = pd.to_datetime(grassland_bird_observations['End_Time']).dt.time


In [166]:
print(type(grassland_bird_observations['Date'].iloc[0]))


<class 'datetime.date'>


In [168]:
print(grassland_bird_observations.columns)


Index(['Plot_Name', 'Date', 'Start_Time', 'End_Time', 'Observer', 'Visit',
       'Interval_Length', 'ID_Method', 'Distance', 'Flyover_Observed', 'Sex',
       'Common_Name', 'Temperature', 'Humidity', 'Initial_Three_Min_Cnt',
       'Sky_Code', 'Wind_Code', 'Disturbance_Code', 'Interval_Start',
       'Interval_End', 'Interval_Avg', 'Distance_Code'],
      dtype='object')


In [ ]:
grassland_bird_observations.to_sql('grassland_bird_observations', con=engine, if_exists='append', index=False)


In [169]:
grassland_bird_observations.rename(columns={
    'Interval_Start': 'Interval_Length_Start',
    'Interval_End': 'Interval_Length_End',
    'Interval_Avg': 'Interval_Length_Avg'
}, inplace=True)


In [170]:
grassland_bird_observations.drop(columns=['Interval_Length', 'Distance'], inplace=True, errors='ignore')


In [171]:
expected_columns = [
    'Plot_Name', 'Date', 'Start_Time', 'End_Time', 'Observer', 'Visit',
    'Interval_Length_Start', 'Interval_Length_End', 'Interval_Length_Avg',
    'ID_Method', 'Distance_Code', 'Flyover_Observed', 'Sex', 'Common_Name',
    'Temperature', 'Humidity', 'Sky_Code', 'Wind_Code',
    'Disturbance_Code', 'Initial_Three_Min_Cnt'
]

grassland_bird_observations = grassland_bird_observations[expected_columns]


In [173]:
grassland_bird_observations.to_sql('grassland_bird_observations', con=engine, if_exists='append', index=False)


6826

In [174]:
print(forest_bird_observations.columns)

Index(['Plot_Name', 'Date', 'Start_Time', 'End_Time', 'Observer', 'Visit',
       'Interval_Length', 'ID_Method', 'Distance', 'Flyover_Observed', 'Sex',
       'Common_Name', 'Temperature', 'Humidity', 'Initial_Three_Min_Cnt',
       'Sky_Code', 'Wind_Code', 'Disturbance_Code', 'Interval_Start',
       'Interval_End', 'Interval_Avg', 'Distance_Code'],
      dtype='object')


In [175]:
forest_bird_observations = forest_bird_observations.rename(columns={
    'Interval_Start': 'Interval_Length_Start',
    'Interval_End': 'Interval_Length_End',
    'Interval_Avg': 'Interval_Length_Avg',
    'Distance_Code': 'Distance_Code'  # same, just ensure exists
})

# Drop columns not in SQL table if needed
forest_bird_observations = forest_bird_observations.drop(columns=['Interval_Length', 'Distance'], errors='ignore')


In [176]:
forest_bird_observations.to_sql('forest_bird_observations', con=engine, if_exists='append', index=False)


8542